In [1]:
import pandas as pd


In [12]:
df=pd.read_csv("transformed_df.csv")

In [3]:
def replace_with_unique(df, column: str):
    uniques = df[column].unique()
    mapping = dict(zip(uniques, range(0, len(uniques))))
    df[column].replace(mapping, inplace=True)
    return df
replace_with_unique(df, "diagnostic")


,age,bleed,grew,itch,elevation,changed,background_mother,background_father,has_sewage_system,region,smoke,gender,pesticide,fitspatrick,hurt,diagnostic,img_id,image_path
0,0.404762,0,0,1,0,0,1,0,0,0,0,1,1,1,0,0,PAT_169_694_411.png,./cancer/all_cancer_images/PAT_169_694_411.png
1,0.511905,0,0,1,0,0,1,1,0,0,0,1,1,1,0,0,PAT_38_1003_68.png,./cancer/all_cancer_images/PAT_38_1003_68.png
2,0.607143,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,PAT_65_101_847.png,./cancer/all_cancer_images/PAT_65_101_847.png
3,0.666667,0,0,1,1,0,1,1,0,0,0,0,0,1,1,0,PAT_69_1053_540.png,./cancer/all_cancer_images/PAT_69_1053_540.png
4,0.809524,1,0,1,1,0,1,1,0,0,0,0,1,1,1,0,PAT_260_400_566.png,./cancer/all_cancer_images/PAT_260_400_566.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,0.726190,0,0,0,0,0,1,1,0,0,1,1,1,0,0,4,PAT_216_331_678.png,./cancer/all_cancer_images/PAT_216_331_678.png
368,0.714286,0,1,1,1,0,1,1,1,0,0,1,0,1,0,4,PAT_365_751_709.png,./cancer/all_cancer_images/PAT_365_751_709.png
369,0.702381,0,1,1,1,1,0,0,1,0,1,1,0,0,0,4,PAT_771_1488_562.png,./cancer/all_cancer_images/PAT_771_1488_562.png
370,0.619048,1,1,1,1,0,1,1,1,0,1,0,0,1,0,4,PAT_319_680_832.png,./cancer/all_cancer_images/PAT_319_680_832.png


In [13]:
import numpy as np
X  = np.array(df.drop(df[["diagnostic","img_id", "image_path"]], axis=1))
y = np.array(df["diagnostic"])

In [14]:
X.shape

(372, 15)

In [15]:
y.shape

(372,)

In [16]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [17]:
y.shape

(372, 5)

In [26]:
### 2.2 Load model
from tensorflow.keras.applications.mobilenet_v2 import (
    MobileNetV2,
    preprocess_input,
    decode_predictions
)

In [27]:
pretrained_model = MobileNetV2(
    weights='imagenet'
)

In [28]:
def load_image(image_path):
    """
    loads an input image into PIL format of size 224 X 224
    """
    img = image.load_img(
        path=image_path,
        target_size=(500,500)
    )
    return img

In [29]:
def batch_image(loaded_image):
    """
    converts a loaded image to a batch
    """
    img_array = image.img_to_array(loaded_image)
    img_batch = np.array([img_array])
    
    return img_batch

In [30]:
def preprocess_image(batched_image):
    """
    preprocesses a batch image to make suitable for mobilenet_v2
    """
    processed_img = preprocess_input(batched_image)
    
    return processed_img

In [31]:
def show_preprocess_image(image_path):
    """
    shows loaded image given an image path, and
    preprocesses it to make suitable for mobilenet_v2
    """
    # load resized image
    loaded_img = load_image(image_path)
    
    # display image
    plt.figure(figsize=(3,3), dpi=100)
    plt.imshow(loaded_img)
    
    # create batched image
    batch_img = batch_image(loaded_img)
    
    # preprocess image
    processed_img = preprocess_image(batch_img)
    
    return processed_img

In [32]:
def image_class_prediction(model, preprocessed_image):
    """
    spits out the top-5 predictions of a given model with a proprocessed image
    """
    model_prediction = model.predict(preprocessed_image,verbose=0)
    decoded_prediction = decode_predictions(model_prediction,top=5)[0]
    data = {
        'image_class': [_[1] for _ in decoded_prediction],
        'probability': [_[2] for _ in decoded_prediction]
    }
    df = pd.DataFrame(data=data)
    return df

In [33]:
CLASSES = transformed_df["diagnostic"].values 

NameError: name 'transformed_df' is not defined

In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_directory = "./cancer/all_cancer_images/"

datagen = ImageDataGenerator(rescale=1./255)

batch_size = 372
target_size = (224, 224) 
class_mode = 'categorical'

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_directory,
    x_col="img_id",
    y_col="diagnostic",
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=True,
)

Found 372 validated image filenames belonging to 5 classes.


In [36]:
xtrain, ytrain = next(train_generator)
xtrain.shape, ytrain.shape

((372, 224, 224, 3), (372, 5))

In [37]:
def BuildModel():
    '''
    Compiles a model by stacking dense layers on top of base model.
    '''
    # Initialize the Sequential API to stack the layers
    model = keras.Sequential()
    
    # Convolutional base 
    model.add(BASE_MODEL)
    
    # Flatten the output of the base model
    model.add(keras.layers.Flatten())
    
    # Add dense layers
    model.add(keras.layers.Dense(units=2048, activation='relu'))
    model.add(keras.layers.Dense(units=512, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    
    # Update the number of classes in the last dense layer
    num_classes = len(train_generator.class_indices)

    # Output layer with softmax activation function
    model.add(keras.layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=[keras.metrics.CategoricalAccuracy()]
    )


    return model


In [ ]:
from keras.layers.merge import concatenate # please share the import next time
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Flatten, Dense, Concatenate, Input
from keras import Model

# Defining model input
input_ = Input(shape=(224, 224, 3))

# Defining first parallel layer
in_1 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(input_)
conv_1 = BatchNormalization()(in_1)
conv_1 = AveragePooling2D(pool_size=(2, 2), strides=(3, 3))(conv_1)

# Defining second parallel layer
in_2 = Conv2D(filters=16, kernel_size=(5, 5), activation='relu', padding='same')(input_)
conv_2 = BatchNormalization()(in_2)
conv_2 = AveragePooling2D(pool_size=(2, 2), strides=(3, 3))(conv_2)


# Concatenating layers
concat = concatenate([conv_1, conv_2])
flat = Flatten()(concat)
out = Dense(units=4, activation='softmax')(flat)

model = Model(inputs=[input_], outputs=[out])
model.summary()

In [43]:
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, concatenate, Input
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers import Dropout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.mobilenet_v2 import (
    MobileNetV2,
    preprocess_input,
    decode_predictions
)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, InputLayer
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50

In [44]:

# Load and preprocess the images using ImageDataGenerator
image_directory = "./cancer/all_cancer_images/"
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 372
target_size = (224, 224)
class_mode = 'categorical'

# Convert "diagnostic" column to string type
transformed_df["diagnostic"] = transformed_df["diagnostic"].astype(str)

train_generator = datagen.flow_from_dataframe(
    dataframe=transformed_df,
    directory=image_directory,
    x_col="img_id",
    y_col="diagnostic",
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=True,
)

Found 372 validated image filenames belonging to 5 classes.


In [45]:
xtrain, ytrain = next(train_generator)

# Define the CNN model
BASE_MODEL = ResNet50(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(224, 224, 3)
)
BASE_MODEL.trainable = False

In [46]:
def BuildModel():
    model = Sequential()
    model.add(BASE_MODEL)
    model.add(Flatten())
    model.add(Dense(units=2048, activation='relu'))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.5))
    num_classes = len(train_generator.class_indices)
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(
        optimizer=Adam(),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
    return model

In [47]:
cnn_model = BuildModel()
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_9 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_10 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 5)                 2565      
                                                                 
Total params: 28,835,717
Trainable params: 5,248,005
N

In [48]:
# Load and preprocess the ANN data
X_ann = np.array(transformed_df.drop(["diagnostic", "img_id", "image_path"], axis=1))
y_ann = np.array(transformed_df["diagnostic"])
label_encoder = LabelEncoder()
y_ann = label_encoder.fit_transform(y_ann)
y_ann = to_categorical(y_ann)

# Define the ANN model
input_ann = Input(shape=(X_ann.shape[1],))
dense1 = Dense(units=15, activation='relu')(input_ann)
dense2 = Dense(units=8, activation='relu')(dense1)
ann_output = Dense(units=5, activation='softmax')(dense2)

ann_model = Model(inputs=[input_ann], outputs=[ann_output])
ann_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 15)]              0         
                                                                 
 dense_12 (Dense)            (None, 15)                240       
                                                                 
 dense_13 (Dense)            (None, 8)                 128       
                                                                 
 dense_14 (Dense)            (None, 5)                 45        
                                                                 
Total params: 413
Trainable params: 413
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Concatenate the models
concatenated = concatenate([cnn_model.output, ann_model.output])
combined_output = Dense(units=5, activation='softmax')(concatenated)

# Create the combined model
combined_model = Model(inputs=[cnn_model.input, ann_model.input], outputs=[combined_output])
combined_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 resnet50_input (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 2048)         23587712    ['resnet50_input[0][0]']         
                                                                                                  
 flatten_2 (Flatten)            (None, 2048)         0           ['resnet50[0][0]']               
                                                                                                  
 dense_9 (Dense)                (None, 2048)         4196352     ['flatten_2[0][0]']        

In [50]:
# Compile and train the combined model
opt = tf.keras.optimizers.RMSprop()
combined_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])
history = combined_model.fit([xtrain, X_ann], y_ann, batch_size=32, epochs=1000, validation_split=0.2)


Epoch 1/1000


2023-06-15 11:58:38.854737: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 178827264 exceeds 10% of free system memory.


10/10 [==============================] - ETA: 0s - loss: 1.5935 - categorical_accuracy: 0.2357

KeyboardInterrupt: 